# DISNEY MOVIES GROSS INCOME

![Uploading disney.jpg](disney.jpg)


The Walt Disney Company, a global entertainment conglomerate, has a long and storied history of producing blockbuster movies. Understanding the revenue trends of Disney movies over time can provide valuable insights into the company's performance, audience preferences, and the overall dynamics of the film industry. In this project, I aim to analyze Disney movie revenue data to identify trends, patterns, and factors influencing revenue generation.

## Exploratory Analysis

This analysis can inform strategic decision-making processes within Disney and help stakeholders better understand the factors driving revenue generation in the entertainment sector.

**List of genres along with the highest-grossing movie.**

In [7]:
SELECT d.genre, d.movie_title, d.total_gross
FROM main.disney_movies_total_gross d
INNER JOIN (
    SELECT genre, MAX(total_gross) AS max_gross
    FROM main.disney_movies_total_gross
    GROUP BY genre
) grouped_genres ON d.genre = grouped_genres.genre AND d.total_gross = grouped_genres.max_gross

,genre,movie_title,total_gross
0,,The Black Hole,35841901
1,Romantic Comedy,Pretty Woman,178406268
2,Musical,Beauty and the Beast,218951625
3,Horror,An American Werewolf in Paris,26570463
4,Thriller/Suspense,The Sixth Sense,293506292
5,Black Comedy,The Royal Tenenbaums,52353636
6,Comedy,Cars,244082982
7,Concert/Performance,Hannah Montana/Miley Cyrus: Best of B…,65281781
8,Documentary,Earth,32011576
9,Action,The Avengers,623279547


**Movies with the Highest Gross Revenue for Each MPAA Rating**

In [8]:
WITH RankedMovies AS (
    SELECT
        movie_title,
        MPAA_rating,
        total_gross,
        RANK() OVER (PARTITION BY MPAA_rating ORDER BY total_gross DESC) AS rank
    FROM main.disney_movies_total_gross
)
SELECT
    movie_title,
    MPAA_rating,
    total_gross
FROM RankedMovies
WHERE rank = 1;

,movie_title,MPAA_rating,total_gross
0,The Lion King,G,422780140
1,The Jungle Book,Not Rated,141843000
2,Finding Dory,PG,486295561
3,The Rescuers,,48775599
4,Star Wars Ep. VII: The Force Awakens,PG-13,936662225
5,Pretty Woman,R,178406268


**General Audience Movies with the highest gross income**

In [14]:
SELECT 
    movie_title, 
    MPAA_rating, 
    total_gross
FROM 
    main.disney_movies_total_gross
WHERE 
    MPAA_rating = 'G'
ORDER BY 
    total_gross DESC;

,movie_title,MPAA_rating,total_gross
0,The Lion King,G,422780140
1,Toy Story 3,G,415004880
2,Finding Nemo,G,380529370
3,"Monsters, Inc.",G,289423425
4,Monsters University,G,268488329
...,...,...,...
81,Fantasia 2000 (Theatrical Release),G,9103630
82,Aliens of the Deep,G,8968684
83,A Beautiful Planet,G,7895708
84,Young Black Stallion,G,6751389


**Movie with the Highest Increase in Total Gross Revenue from the Previous Year**

In [15]:
WITH ranked_movies AS (
  SELECT
    movie_title,
    release_date,
    genre,
    MPAA_rating,
    total_gross,
    inflation_adjusted_gross,
    LAG(total_gross) OVER (ORDER BY release_date) AS previous_year_gross,
    total_gross - LAG(total_gross) OVER (ORDER BY release_date) AS gross_increase
  FROM main.disney_movies_total_gross
)

SELECT
  movie_title,
  release_date,
  genre,
  MPAA_rating,
  total_gross,
  inflation_adjusted_gross,
  previous_year_gross,
  gross_increase
FROM ranked_movies
WHERE gross_increase IS NOT NULL
ORDER BY gross_increase DESC
LIMIT 1;

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,previous_year_gross,gross_increase
0,Star Wars Ep. VII: The Force Awakens,2015-12-18,Adventure,PG-13,936662225,936662225,172062763,764599462


**Movies with the Biggest Difference Between Total Gross and Inflation Adjusted Gross**

In [16]:
SELECT movie_title, total_gross, inflation_adjusted_gross, 
       (total_gross - inflation_adjusted_gross) AS gross_difference
FROM main.disney_movies_total_gross
ORDER BY gross_difference DESC;


,movie_title,total_gross,inflation_adjusted_gross,gross_difference
0,The Many Adventures of Winnie the Pooh,0,0,0
1,Amy,0,0,0
2,Condorman,0,0,0
3,Frank McKlusky C.I.,0,0,0
4,Jonas Brothers: The 3D Concert Experi…,38174685,38174685,0
...,...,...,...,...
574,Lady and the Tramp,93600000,1236035515,-1142435515
575,101 Dalmatians,153000000,1362870985,-1209870985
576,Fantasia,83320000,2187090808,-2103770808
577,Pinocchio,84300000,2188229052,-2103929052


**Identifying Movies with the Highest and Lowest Total Gross Revenue for Each Genre**

In [2]:
SELECT 
    genre,
    MAX(movie_title) AS highest_grossing_movie,
    MAX(total_gross) AS highest_total_gross,
    MIN(movie_title) AS lowest_grossing_movie,
    MIN(total_gross) AS lowest_total_gross
FROM main.disney_movies_total_gross
GROUP BY genre;

,genre,highest_grossing_movie,highest_total_gross,lowest_grossing_movie,lowest_total_gross
0,Musical,Tim Burton's The Nightmare Before Chr…,218951625,Babes in Toyland,5632086
1,Horror,The Puppet Masters,26570463,An American Werewolf in Paris,4877567
2,Documentary,X Games 3D: The Movie,32011576,A Beautiful Planet,20521
3,Concert/Performance,Jonas Brothers: The 3D Concert Experi…,65281781,Hannah Montana/Miley Cyrus: Best of B…,38174685
4,Adventure,Zootopia,936662225,1492: Conquest of Paradise,2815
5,Drama,crazy/beautiful,201151353,25th Hour,0
6,Comedy,You Again,244082982,101 Dalmatians,45779
7,,The War at Home,35841901,A Gun in Betty Lou's Handbag,0
8,Action,Tron,623279547,3 Ninjas,0
9,Romantic Comedy,While You Were Sleeping,178406268,A Pyromaniac's Love Story,468240


**Identifying movies with the longest movie title**

In [24]:
SELECT 
    movie_title,
    LENGTH(movie_title) AS title_length
FROM main.disney_movies_total_gross
ORDER BY title_length DESC;


,movie_title,title_length
0,The Princess Diaries 2: Royal Engagement,40
1,The Chronicles of Narnia: Prince Caspian,40
2,The Many Adventures of Winnie the Pooh,38
3,DuckTales: The Movie - Treasure of th…,38
4,Homeward Bound: The Incredible Journey,38
...,...,...
574,Bolt,4
575,Prom,4
576,Amy,3
577,Tex,3


**Identifying the Top 5 Highest Grossing Movies in Each Genre**

In [2]:
SELECT genre,
    movie_title,
	RIGHT(release_date, 4) AS release_year,
    total_gross,
    RANK() OVER (PARTITION BY genre ORDER BY total_gross DESC) AS genre_rank
FROM main.disney_movies_total_gross
WHERE genre IS NOT NULL AND genre != ''
ORDER BY genre, genre_rank, release_year;

,genre,movie_title,release_year,total_gross,genre_rank
0,Action,The Avengers,2012,623279547,1
1,Action,Avengers: Age of Ultron,2015,459005868,2
2,Action,Iron Man 3,2013,408992272,3
3,Action,Captain America: Civil War,2016,408084349,4
4,Action,Captain America: The Winter Soldier,2014,259746958,5
...,...,...,...,...,...
557,Western,Open Range,2003,58331254,3
558,Western,Shanghai Noon,2000,56932305,4
559,Western,Tombstone,1993,56505065,5
560,Western,The Alamo,2004,22406362,6
